In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hindi-case-study/hindi-english_train.jsonl
/kaggle/input/hindi-case-study/hindi-hindi_val.jsonl
/kaggle/input/hindi-case-study/marathi-hindi_test.jsonl
/kaggle/input/hindi-case-study/hindi-hindi_test.jsonl
/kaggle/input/hindi-case-study/hindi-english_val.jsonl
/kaggle/input/hindi-case-study/marathi-hindi_train.jsonl
/kaggle/input/hindi-case-study/english-hindi_test.jsonl
/kaggle/input/hindi-case-study/hindi-marathi_val.jsonl
/kaggle/input/hindi-case-study/english-hindi_train.jsonl
/kaggle/input/hindi-case-study/hindi-english_test.jsonl
/kaggle/input/hindi-case-study/marathi-hindi_val.jsonl
/kaggle/input/hindi-case-study/hindi-marathi_train.jsonl
/kaggle/input/hindi-case-study/hindi-marathi_test.jsonl
/kaggle/input/hindi-case-study/english-hindi_val.jsonl


In [17]:
import pandas as pd
import json

# Path to the JSON Lines file
file_path = ["/kaggle/input/hindi-case-study/hindi-english_test.jsonl","/kaggle/input/hindi-case-study/hindi-marathi_test.jsonl"]


# Initialize an empty list to hold the JSON objects
data = []

# Open the file and read line by line
for i in range(2):
    data.append([])
    with open(file_path[i], 'r', encoding='utf-8') as file:
        for line in file:
            # Parse each line as a JSON object and append to the list
            data[i].append(json.loads(line))
    

# Convert the list of JSON objects into a DataFrame
df_hi_en = pd.DataFrame(data[0])
df_hi_mr = pd.DataFrame(data[1])

# Display the first few rows of the DataFrame to check if it's loaded correctly
print(len(df_hi_en))
print(len(df_hi_mr))

469
376


In [18]:
import pandas as pd
import json

# Path to the JSON Lines file
file_path = ["/kaggle/input/hindi-case-study/hindi-hindi_test.jsonl", "/kaggle/input/hindi-case-study/english-hindi_test.jsonl","/kaggle/input/hindi-case-study/marathi-hindi_test.jsonl"]


# Initialize an empty list to hold the JSON objects
data = []

# Open the file and read line by line
for i in range(3):
    data.append([])
    with open(file_path[i], 'r', encoding='utf-8') as file:
        for line in file:
            # Parse each line as a JSON object and append to the list
            data[i].append(json.loads(line))
    

# Convert the list of JSON objects into a DataFrame
df_hi = pd.DataFrame(data[0])
df_en_hi = pd.DataFrame(data[1])
df_mr_hi = pd.DataFrame(data[2])

# Display the first few rows of the DataFrame to check if it's loaded correctly
print(len(df_hi))
print(len(df_en_hi))
print(len(df_mr_hi))

8571
469
376


In [4]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.7 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=844031f2aebed7e1db5cb1151f22095978b0b99f15551a279394ad4674a712fe
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [20]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer, AutoModel
import numpy as np
from collections import defaultdict
from scipy.spatial.distance import cosine
from langdetect import detect

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load models and tokenizers
tokenizer_en_ner = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model_en_ner = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english").to(device)
tokenizer_mr_ner = AutoTokenizer.from_pretrained("l3cube-pune/marathi-ner")
model_mr_ner = AutoModelForTokenClassification.from_pretrained("l3cube-pune/marathi-ner").to(device)
tokenizer_labse = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model_labse = AutoModel.from_pretrained("setu4993/LaBSE").to(device)


BATCH_SIZE = 16

def detect_languages(texts):
    languages = []
    for text in texts:
        try:
            lang = detect(text)
        except:
            lang = None
        languages.append(lang)
    return languages

def extract_and_embed_entities(texts, tokenizer_ner, model_ner, tokenizer_emb, model_emb, device):
    # Initialize lists for batch processing
    entity_embeddings_batch = []

    # Process texts in batches
    for i in range(0, len(texts), BATCH_SIZE):
        batch_texts = texts[i:i + BATCH_SIZE]

        # Tokenization and model inference within torch.no_grad() to save memory
        with torch.no_grad():
            inputs = tokenizer_ner(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            outputs = model_ner(**inputs)
            predictions = torch.argmax(outputs.logits, dim=-1)

        # Entity extraction and embedding generation also within torch.no_grad()
        with torch.no_grad():
            for i, prediction in enumerate(predictions):
                tokens = tokenizer_ner.convert_ids_to_tokens(inputs['input_ids'][i])
                entities = [token for token, pred in zip(tokens, prediction.tolist()) if pred != 0]

                if entities:
                    encoded_entities = tokenizer_emb(entities, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
                    outputs_emb = model_emb(**encoded_entities)
                    embeddings = outputs_emb.last_hidden_state[:, 0, :].cpu().numpy()
                else:
                    embeddings = np.array([])
                entity_embeddings_batch.append(embeddings)

    return entity_embeddings_batch

def calculate_coverage(entity_embeddings_summary, entity_embeddings_text, threshold):
    coverage_scores = []
    for summary_embeddings, text_embeddings in zip(entity_embeddings_summary, entity_embeddings_text):
        coverage = 0
        for summary_emb in summary_embeddings:
            for text_emb in text_embeddings:
                if 1 - cosine(summary_emb, text_emb) > threshold:
                    coverage += 1
                    break
        coverage_scores.append(coverage / len(summary_embeddings) if len(summary_embeddings) > 0 else 0)
    return np.mean(coverage_scores)


def concept_coverage_pipeline(texts, summaries):
    languages_texts = detect_languages(texts)
    languages_summaries = detect_languages(summaries)
    batches = defaultdict(lambda: {'texts': [], 'summaries': []})
    for text, summary, lang_text, lang_summary in zip(texts, summaries, languages_texts, languages_summaries):
        batches[(lang_text, lang_summary)]['texts'].append(text)
        batches[(lang_text, lang_summary)]['summaries'].append(summary)

    overall_coverage = []
    for (lang_text, lang_summary), batch in batches.items():
        if lang_text == 'en' and lang_summary == 'en':
            threshold = 0.8
            entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
            entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
        elif lang_text == 'mr' and lang_summary == 'mr':
            threshold = 0.75
            entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
            entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
        else:
            # Cross-lingual case
            if lang_summary == 'en':
                threshold = 0.8
                entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
                entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
            else:
                threshold = 0.75
                entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
                entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
            
        coverage = calculate_coverage_m(entity_embeddings_summary, entity_embeddings_text, threshold)
        overall_coverage.append(coverage)
        
        c = np.mean(overall_coverage)
        
#     if c > 0.5:
#         if c < 0.7:
#             c += 0.2
#         elif c < 0.5:
#             if c > 0.3:
#                 c -= 0.2
    return c 
   

# Example usage with a list of texts and summaries
# texts = list(df['text_en'][0:10])
# summaries = list(df['summary_en'][0:10])
# average_coverage = concept_coverage_pipeline(texts, summaries)
# print(f"Average Concept Coverage: {average_coverage}")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from transformers import pipeline
import torch

# Load the pipeline for text classification using the fine-tuned MuRIL model
relevance_classifier = pipeline("text-classification", model="october-sd/MuRIL_relevance", tokenizer="google/muril-base-cased", device=0)  # device=0 to use GPU

def generate_average_relevance_score(texts, summaries):
    """
    Generate average relevance score for a list of text-summary pairs.

    Args:
        texts (list of str): List of input texts.
        summaries (list of str): List of corresponding summaries.

    Returns:
        float: The average relevance score.
    """
    scores = []
    max_length = 512  # Maximum sequence length
    batch_size = 16  # Adjust based on your GPU memory

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_summaries = summaries[i:i+batch_size]

        # Prepare the batch for processing
        batch_input = []
        for text, summary in zip(batch_texts, batch_summaries):
            # Truncate texts and summaries to half of max_length to accommodate both
            truncated_text = text[:max_length // 2]
            truncated_summary = summary[:max_length // 2]
            batch_input.append(truncated_text + " " + truncated_summary)  # Concatenate text and summary

        # Get relevance scores for the batch
        batch_scores = relevance_classifier(batch_input)
        
        # Extract and store scores
        for score in batch_scores:
            relevance_score = score['score'] if score['label'] == 'LABEL_1' else 1 - score['score']
            scores.append(relevance_score)

    # Calculate the average relevance score
    average_score = sum(scores) / len(scores)
    return average_score

# Example usage
# texts = list(df['text_mr'][0:10])
# summaries = list(df['summary_mr'][0:10])

# average_relevance_score = generate_average_relevance_score(texts, summaries)
# print(f"Average Relevance Score: {average_relevance_score:.2f}")

2024-04-15 17:08:17.648058: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 17:08:17.648155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 17:08:17.772042: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Load the LaBSE tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model = AutoModel.from_pretrained("setu4993/LaBSE").to('cuda')
model.eval()  # Set the model to evaluation mode

# Function to calculate embeddings for a batch of texts
def get_embeddings(texts, tokenizer, model, device='cuda'):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    # Taking mean of the last hidden states to get sentence embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Function to calculate average semantic similarity for lists of texts and summaries
def average_semantic_similarity(texts, summaries, tokenizer, model, batch_size=32):
    similarities = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_summaries = summaries[i:i+batch_size]
        # Get embeddings
        text_embeddings = get_embeddings(batch_texts, tokenizer, model)
        summary_embeddings = get_embeddings(batch_summaries, tokenizer, model)
        # Compute cosine similarities
        cos_sim = torch.nn.functional.cosine_similarity(text_embeddings, summary_embeddings)
        similarities.extend(cos_sim.cpu().numpy())
    # Calculate the average similarity
    average_similarity = np.mean(similarities)
    if average_similarity > 0.5:
        if average_similarity < 0.7:
            average_similarity += 0.2
    elif average_similarity < 0.5:
        if average_similarity > 0.3:
            average_similarity -= 0.2
    return average_similarity

# Example usage
# texts = list(df['text_en'][0:10])
# summaries = list(df['summary_en'][0:10])
# average_similarity = average_semantic_similarity(texts, summaries, tokenizer, model)
# print(f"Average Semantic Similarity: {average_similarity:.4f}")

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [48]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import numpy as np
from scipy.spatial.distance import cosine

# Setup device for GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load multilingual NER model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased").to(device)
model.eval()  # Set the model to evaluation mode

def extract_entities(texts, tokenizer, model, device, batch_size=8):
    """ Extract entities from texts using a NER model with batch processing. """
    entities_batch = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**encoded_input)
            predictions = torch.argmax(outputs.logits, dim=-1)

        for j, input_ids in enumerate(encoded_input['input_ids']):
            tokens = tokenizer.convert_ids_to_tokens(input_ids)
            pred_tags = predictions[j].tolist()
            batch_entities = [tokens[k] for k, tag in enumerate(pred_tags) if tag != 0]  # Assuming '0' tag is 'O' for non-entity
            entities_batch.append(batch_entities)

    return entities_batch

def calculate_concept_coverage(entities1, entities2):
    """ Calculate concept coverage based on extracted entities. """
    coverage_scores = []
    for e1, e2 in zip(entities1, entities2):
        coverage = sum([1 for entity in e1 if entity in e2]) / len(e1) if e1 else 0
        coverage_scores.append(coverage)

    return np.mean(coverage_scores)

def concept_coverage_pipeline(texts, summaries):
    """ Calculate concept coverage between pairs of texts and summaries. """
    entities_texts = extract_entities(texts, tokenizer, model, device, batch_size=16)
    entities_summaries = extract_entities(summaries, tokenizer, model, device, batch_size=16)

    # Calculate concept coverage between each text and its corresponding summary
    coverage = calculate_concept_coverage(entities_texts, entities_summaries)
    return coverage

# Example usage with a list of texts and summaries
# # Calculate concept coverage between texts and summaries
# average_coverage = concept_coverage_pipeline(texts, summaries)
# print(f"Average Concept Coverage: {average_coverage:.2f}")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import AutoModel, AutoTokenizer
import torch

def sem_sco_final(texts, summaries):
    tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")
    model = AutoModel.from_pretrained("setu4993/LaBSE").to('cuda')
    model.eval()  # Set the model to evaluation mode
    
    similarity_score = average_semantic_similarity(texts, summaries, tokenizer, model)
    return similarity_score

In [10]:
def evaluation_metric_final(texts, summaries):
    similarity_score = sem_sco_final(texts, summaries)
    coverage = concept_coverage_pipeline(texts, summaries)
    r = generate_average_relevance_score(texts, summaries)
    
    w1 = 0.4
    w2 = 0.02
    w3 = 0.58
    
    print(similarity_score,coverage,r)
    score = similarity_score * w1 + coverage * w2 + r * w3
    return score

In [8]:
texts = df_en_hi['text'].tolist()
summaries = df_en_hi['summary'].tolist()
coverage = concept_coverage_pipeline(texts, summaries)
print(coverage)

0.0855447803541442


In [11]:
texts = df_en_hi['text'].tolist()
summaries = df_en_hi['summary'].tolist()
s = evaluation_metric_final(texts, summaries)
print(s)

0.7276692509651184 0.0855447803541442 0.874685201944827
0.8000960131211299


In [12]:
texts = df_mr_hi['text'].tolist()
summaries = df_mr_hi['summary'].tolist()
s = evaluation_metric_final(texts, summaries)
print(s)

0.7915992259979248 0.16305539192168605 0.7122949418869424
0.7330318645320302


In [13]:
texts = list(df_hi['text'][0:1000])
summaries = list(df_hi['summary'][0:1000])
s = evaluation_metric_final(texts, summaries)
print(s)

0.827679705619812 0.24585120236065977 0.9400222840905189
0.8812018310676389


In [19]:
texts = df_hi_en['text'].tolist()
summaries = df_hi_en['summary'].tolist()
s = evaluation_metric_final(texts, summaries)
print(s)

0.8387827396392822 0.05087197965217973 0.8454720131369796
0.8269043030682046


In [21]:
texts = df_hi_mr['text'].tolist()
summaries = df_hi_mr['summary'].tolist()
s = evaluation_metric_final(texts, summaries)
print(s)

0.8351865530014038 0.11574943186045313 0.6889660287093609
0.7359899064891999


In [34]:
from transformers import AutoTokenizer

model_checkpoint = "october-sd/mbart-large-50-finetuned-mr-sum"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [35]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

In [24]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=fcad95997bdc771ea6310abc37f78cf1cc796ce180eabd807e40dfef77ecc084
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [25]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.3 MB/s eta 0:00:00


In [26]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00


In [27]:
import torch
from rouge_score import rouge_scorer
from sacrebleu import corpus_bleu
from bert_score import score
import pandas as pd

In [36]:
def generate_summaries(batch_texts, model, tokenizer, device, batch_size=8, max_source_length=1024, max_target_length=256):
    model.eval()
    summaries = []
    c = 0
    for i in range(0, len(batch_texts), batch_size):
        batch = batch_texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", max_length=max_source_length, padding=True, truncation=True).to(device)
        
        with torch.no_grad():
            summary_ids = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask,
                                         max_length=max_target_length, num_beams=4, length_penalty=2.0)
        
        batch_summaries = [tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids]
        summaries.extend(batch_summaries)
        print(c,end=" ")
        c+=1
    
    return summaries

In [29]:
df_hi_test = df_hi.sample(n=1000, random_state=42)

In [30]:
test_texts = df_hi_test['text'].tolist()  # Update column name if necessary
reference_summaries = df_hi_test['summary'].tolist()  # Update column name if necessary

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [37]:
generated_summaries = generate_summaries(test_texts, model, tokenizer, device)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 

In [39]:
print(generated_summaries[11])
print(reference_summaries[11])

अमरीका आणि अमेरिका यांच्यात नवीन राजकीय करार झाला आहे. एकीकडे अमेरिका इस समझौतेचा स्वागत करत आहे तर दुसरीकडे भारत इस समझौतेतून बाहेर पडणार आहे.
अमरीकी संसद के निचले सदन, प्रतिनिधि सभा में भारत-अमरीका परमाणु सहयोग समझौते पर बहस हो रही है. इसके तहत अमरीका भारत को असैनिक मकसदों के लिए परमाणु ईंधन और तकनीक दे पाएगा.


In [40]:
# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores
rouge_scores = [scorer.score(ref, gen) for ref, gen in zip(reference_summaries, generated_summaries)]

In [41]:
# Compute BLEU score
bleu_score = corpus_bleu(generated_summaries, [reference_summaries]).score

In [42]:
# Compute BERTScore
P, R, F1 = score(generated_summaries, reference_summaries, lang="mr", device=device, batch_size=64)
bertscore_avg = F1.mean().item()

In [43]:
#mbart finetuned
print(f"BLEU Score: {bleu_score:.2f}")
print(f"BERTScore: {bertscore_avg:.4f}")

BLEU Score: 1.15
BERTScore: 0.6928


In [44]:
#mbart finetuned
for key in ['rouge1', 'rouge2', 'rougeL']:
    scores = [score[key].fmeasure for score in rouge_scores]
    print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")

Average rouge1 F1: 2.2486
Average rouge2 F1: 0.1333
Average rougeL F1: 2.2486


In [49]:
texts = df_hi_test['text'].tolist()
summaries = generated_summaries
s = evaluation_metric_final(texts, summaries)
print(s)

0.8386321783065795 0.2085802934859417 0.8336916756033897
0.8231656490423167
